In [1]:
import chardet
import pandas as pd

pd.set_option('display.max_columns', None)

with open('/Users/heogeonho/Univ/Data Mining/데이터 전처리/2_김시형 (2).csv', 'rb') as f:
    result = chardet.detect(f.read())
    encoding = result['encoding']

data = pd.read_csv('/Users/heogeonho/Univ/Data Mining/데이터 전처리/2_김시형 (2).csv', encoding=encoding)
df = data

df.head(3)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/heogeonho/Univ/Data Mining/데이터 전처리/2_김시형 (2).csv'

In [ ]:
df = df.drop(columns=['읍면동구분명', '번지', '도로명','발화요인소분류명'])
df = df.rename(columns={'Unnamed: 0': 'ID'})

In [ ]:
#화재발생일자
df['화재발생일자'] = pd.to_datetime(df['화재발생일자'], format='%Y%m%d')

#화재발생시간
df['화재발생시간'] = df['화재발생시간'].astype(str).str.zfill(6)
df['화재발생시간'] = df['화재발생시간'].str[:2] + ':' + df['화재발생시간'].str[2:4] + ':' + df['화재발생시간'].str[4:]
df['화재발생시간'] = pd.to_datetime(df['화재발생시간'], format='%H:%M:%S')

#화재발생시간대
df['화재발생시간대'] = pd.cut(df['화재발생시간'].dt.hour, bins=8, labels=['0~3', '3~6', '6~9', '9~12', '12~15', '15~18', '18~21', '21~24'])

df['화재발생시간'] = df['화재발생시간'].dt.strftime('%H:%M:%S')

In [ ]:
df['요일'] = df['화재발생일자'].dt.weekday  # 월=0, 화=1, ..., 일=6 매핑

In [ ]:
df['화재발생_연도'] = df['화재발생일자'].dt.year
df['화재발생_월'] = df['화재발생일자'].dt.month
df['화재발생_일'] = df['화재발생일자'].dt.day

In [ ]:
def merge_reasons(row):
    if row['발화요인대분류명'] == '부주의' or row['발화원인'] == '입산자실화' or row['발화원인'] == '성묘객' or row['발화원인'] == '담뱃불':
        return '인간의 부주의'
    
    elif row['발화요인대분류명'] == '전기적요인' or row['발화요인대분류명'] == '교통사고' or row['발화요인대분류명'] == '화학적요인' or row['발화요인대분류명'] == '기계적요인':
        return '기계/전기/화학 요인'

    elif ( row['발화원인'] == '미상' or row['발화원인'] == '기타' or row['발화원인'] == '건물') and ( row['발화요인대분류명'] == '미상' or row['발화요인대분류명'] == '기타' or row['발화요인대분류명'] == '비화'):
        return '미상'
    
    elif row['발화요인대분류명'] == '방화' or row['발화요인대분류명'] == '방화의심':
        return '방화 및 방화의심'
    
    elif row['발화요인대분류명'] == '자연적인요인':
        return '자연적 요인'
    else:
        return '기타'
    
df['발화종합'] = df.apply(merge_reasons, axis=1)

In [ ]:
count = (df['발화종합'] == '기타').sum()
count

In [ ]:
# 컬럼 이름들을 출력합니다.
print(df.columns)

In [ ]:
df = df.reindex(columns=['ID', '화재발생일자', '화재발생시간', '화재발생_연도', '화재발생_월', '화재발생_일', '요일', '화재발생시간대', '시도명', '시군구명', '읍면동명', '발화지점', '날씨', '온도',
       '습도', '시간단위풍속', '특보내용',
        '발화종합', '발화원인', '발화요인대분류명', '접수일자', '접수시각',
       '소방서명', '서센터명', '소방지역대명', '출동일자', '출동시각', '현장도착일자', '현장도착시각',
       '출동소요시간\t', '현장소방서거리', '현장안전센터거리', '현장소방지역대거리', '초진일자', '초진시각',
       '전체인력수합계', '완진일자', '완진시간', '화재진압시간', '동원장비수', '인명피해수', '재산피해금액', '피해면적'])
df

In [ ]:
df.to_csv('4_허건호01.csv', index=False, encoding='utf-8-sig')
df.to_excel('4_허건호01.xlsx')